In [1]:
import co.uk.bnikolic.qlw.{property_t, qlw, StringVector, LongVector, PropertyVector};
import java.util.UUID;
def UU(s: String) = s+UUID.randomUUID();

// Null value for optional parameters 
def ONull() = qlw.OH_NULL();

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1544455301388_0002,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
import co.uk.bnikolic.qlw.{property_t, qlw, StringVector, LongVector, PropertyVector}
import java.util.UUID
UU: (s: String)String
ONull: ()co.uk.bnikolic.qlw.property_t


In [2]:
// Valuation of Interest Rate Derivatives. See the Excel spreadsheet
// for context.
def PV(sigma: Double) : Double = {

  var yc=qlw.qlFlatForward(UU("flatfowrard"),
    ONull(),
    new property_t("NullCalendar"),
    new property_t(0.044),
    ONull(),
    new property_t("Continuous"),
    new property_t("Annual"))

  var vc=qlw.qlConstantSwaptionVolatility(UU("vol"),
    new property_t(0),
    "TARGET", "f",
    new property_t(sigma),
    ONull()
  )

  var refDate = qlw.qlSettingsEvaluationDate();
  var calendar="TARGET"
  var pv=new PropertyVector()
  pv.push_back(new property_t(refDate))

  var adjustedRefDate=qlw.qlCalendarAdjust(calendar, pv, new property_t("f"))

  var pv2=new PropertyVector()
  pv2.push_back(new property_t("2d"))

  var effectiveDate=qlw.qlCalendarAdvance(calendar
    , new property_t(adjustedRefDate.get(0))
    , pv2
    , ONull()
    , ONull() ).get(0)

  var pv3=new PropertyVector()
  pv3.push_back(new property_t("10Y"))

  var terminationDate=qlw.qlCalendarAdvance(calendar
    , new property_t(effectiveDate)
    , pv3
    , new property_t("Unadjusted")
    , ONull() ).get(0)

  var sFixed=qlw.qlSchedule(UU("schedule")
    , new property_t(effectiveDate+365)
    , new property_t(terminationDate)
    , "1Y"
    , new property_t(calendar)
    , new property_t("Modified Following")
    , new property_t("Modified Following")
    , new property_t("Backward")
    , ONull()
    , ONull()
    , ONull())

  var sFloating=qlw.qlSchedule(UU("schedule")
    , new property_t(effectiveDate+365)
    , new property_t(terminationDate)
    , "6M"
    , new property_t(calendar)
    , new property_t("Modified Following")
    , new property_t("Modified Following")
    , new property_t("Backward")
    , ONull()
    , ONull()
    , ONull())

  var ibor=qlw.qlEuribor(UU("IBOR"),
    "6M",
    new property_t(yc))

  var vanswap=qlw.qlVanillaSwap(UU(""),
    new property_t("Payer"),
    new property_t(1e6),
    sFixed,
    new property_t(0.05),
    "30/360 (Bond Basis)",
    sFloating,
    ibor,
    new property_t(0.0),
    "Actual/360",
    ONull())


  var pe= qlw.qlBlackSwaptionEngine(UU("PE"), yc, vc)

  var ee=qlw.qlEuropeanExercise(UU(""), new property_t("1Y"))

  var ii=qlw.qlSwaption(UU(""),
    vanswap,
    ee,
    "Physical")

  qlw.qlInstrumentSetPricingEngine(ii, pe)

  var NPV=qlw.qlInstrumentNPV(ii)

  // Clean-up all created objects
  qlw.ohRepositoryDeleteAllObjects(ONull())
  NPV;
}


VBox()

PV: (sigma: Double)Double


In [3]:
// Run one valuation as a demo
PV(0.1)

VBox()

res6: Double = 2503.7886048482196


In [7]:
//Use Spark to distribute and run 10k valuations for different volatilites
var result=spark.range(0,10000).map(r => (r,PV(0.0001*r))).toDF()
result.show(10000, false)

VBox()

result: org.apache.spark.sql.DataFrame = [_1: bigint, _2: double]
+----+----------------------+
|_1  |_2                    |
+----+----------------------+
|0   |1.3595803366932583E-4 |
|1   |-5.123754624924515E-4 |
|2   |5.630507615261321E-4  |
|3   |7.330572688575305E-4  |
|4   |-6.607177784006572E-4 |
|5   |-2.824995386345964E-4 |
|6   |-5.540440857081141E-4 |
|7   |-3.078134298774829E-4 |
|8   |3.8706168525775545E-4 |
|9   |-3.379852902196111E-4 |
|10  |-9.026555260209624E-4 |
|11  |-6.464010728957007E-4 |
|12  |2.0268881169458472E-4 |
|13  |4.2660864792396663E-4 |
|14  |-2.4438038569640265E-4|
|15  |-7.99991716564862E-5  |
|16  |-1.7731866694155308E-4|
|17  |-1.0198063340926362E-5|
|18  |1.0573411850955104E-4 |
|19  |-4.4206112200611896E-4|
|20  |7.416730825603964E-4  |
|21  |-4.28446109896262E-4  |
|22  |2.9826109013383526E-4 |
|23  |9.577511422495591E-4  |
|24  |8.345762627963358E-4  |
|25  |8.637369579209042E-4  |
|26  |-9.199861509019027E-4 |
|27  |4.6291802119871476E-4 |
|28 